In [9]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import cv2

from tensorflow.contrib.layers import fully_connected
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)



Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [17]:
n_inputs=784
n_hidden1=392
n_hidden2=196
n_hidden3=n_hidden1
n_outputs=n_inputs

learning_rate=0.01

epochs=100
batch_size=256
display_step=5


In [34]:

weights = { 
            'whid1': tf.Variable(tf.random_normal([n_inputs, n_hidden1])),
            
            'whid2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
            
            'whid3': tf.Variable(tf.random_normal([n_hidden2, n_hidden1])),
                        
            'out': tf.Variable(tf.random_normal([n_hidden1, n_outputs]))}



biases = {  'bhid1': tf.Variable(tf.random_normal([n_hidden1])),
            'bhid2': tf.Variable(tf.random_normal([n_hidden2])),
            'bhid3': tf.Variable(tf.random_normal([n_hidden1])),
            'out': tf.Variable(tf.random_normal([n_outputs]))}


In [13]:

weights1 = { # 5x5 conv, 1 input, and 32 outputs
            'whid1': tf.Variable(tf.random_normal([5, 5, 1, n_hidden1])),
            # 5x5 conv, 32 inputs, and 64 outputs
            'whid2': tf.Variable(tf.random_normal([5, 5, n_hidden1, n_hidden2])),
            # fully connected, 7*7*64 inputs, and 1024 outputs
            'whid3': tf.Variable(tf.random_normal([n_hidden2, n_hidden3])),
                        # 1024 inputs, 10 outputs for class digits
            'out': tf.Variable(tf.random_normal([n_hidden3, n_outputs]))}



biases1 = {  'bhid1': tf.Variable(tf.random_normal([n_hidden1])),
            'bhid2': tf.Variable(tf.random_normal([n_hidden2])),
            'bhid3': tf.Variable(tf.random_normal([n_hidden3])),
            'out': tf.Variable(tf.random_normal([n_outputs]))}



In [14]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1],padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)
'''
def conv2d_transpose(x, W, b, strides=1):
    x=tf.nn.conv2d_transpose(x,W, output_shape=[strides=[1, strides, strides, 1], padding='VALID' )
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)
''' 
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k,1], padding='SAME')


In [4]:
#x=tf.reshape(x, shape=[-1, 28, 28, 1])

def convnet(x, W, b):
    
    #up convolution 1
    uconv1=conv2d(x, weights['whid1'], biases['bhid1'])
    #maxpooling
    uconv1=maxpool2d(uconv1)
    #up convolution 2
    uconv2=conv2d(uconv1, weights['whid2'], biases['bhid2'])
    #maxpooling
    uconv2=maxpool2d(uconv2)
    #downconvolution 1
    dconv1=tf.nn.conv2d_transpose(uconv2, weights['whid3'],output_shape=[n_hidden2, n_hidden3], 
                                    strides=[1,strides, strides,1], padding='VALID')
    
    dconv1=tf.nn.bias_add(dconv1, biases['bhid3'])
    
    dconv1=tf.nn.relu(out)
    
    dconv1=maxpool2d(dconv1)
    
    
    #dowconvolution 2
    out=tf.nn.conv2d_transpose(dconv1, weights['out'],output_shape=[n_hidden3, n_outputs], 
                                    strides=[1,strides, strides,1], padding='VALID')
    out=tf.nn.bias_add(x, biases['out'])
    out=tf.nn.relu(out)
    
    out=maxpool2d(out)
    
    return out




In [38]:
X=tf.placeholder(tf.float32, shape=[None, n_inputs])

In [39]:
def a_e(x):
    uconv1=tf.nn.sigmoid(tf.add(tf.matmul(x, weights['whid1']), biases['bhid1']))
    
    uconv2=tf.nn.sigmoid(tf.add(tf.matmul(uconv1, weights['whid2']), biases['bhid2']))
    
    dconv1=tf.nn.sigmoid(tf.add(tf.matmul(uconv2, weights['whid3']), biases['bhid3']))
    
    out=tf.nn.sigmoid(tf.add(tf.matmul(dconv1, weights['out']), biases['out']))
    

In [40]:
output=a_e(X)

In [45]:
y_pred=output
y_true=X

loss = tf.reduce_mean(y_true - y_pred)
optimizer = tf.train.AdamropOptimizer(learning_rate).minimize(loss)

ValueError: None values not supported.

In [44]:
'''
loss=tf.reduce_mean(tf.square(out-x))

learning_rate=0.01

#cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

optimizer =tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
#train=optimizer.minimise
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
'''

NameError: name 'out' is not defined

In [ ]:

saver=tf.train.Saver()
init = tf.global_variables_initializer()
   
    
epochs=5    
sess.run(init)
batch_size=500
for epoch_i in range(epochs):
    num_batches=mnist.train.num_examples // batch_size
    for batch_i in range(num_batches):
        batch_x, batch_y=mnist.train.next_batch(batch_size)
        sess.run(train, feed_dict={x:batch_x})
    
    training_loss=loss.eval(feed_dict={x:batch_x})
    print("EPOCH: {}". format(epoch, training_loss))
    
saver.save('./model_save/latesmodel.ckpt')